# **TEST THE MODEL 1**

## Installed Libraries

In [ ]:
!pip install --quiet --upgrade pip
!pip uninstall --quiet tensorflow tensorflow-tensorboard tensorflow-estimator
!pip install --quiet gast==0.2.2
!pip install --quiet tensorflow-gpu

# Reference: https://stackoverflow.com/questions/69027356/importing-tensorflow-shows-errors

# for avoiding error: module 'tensorflow_core.keras.activations' has no attribute 'swish'
!pip3 install --quiet --upgrade tensorflow-gpu

Proceed (Y/n)? Y
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-probability 0.16.0 requires gast>=0.3.2, but you have gast 0.2.2 which is incompatible.


In [ ]:
!pip install --quiet transformers==4.1.1
!pip install --quiet tokenizers==0.9.4
!pip install --quiet tensorflow --ignore-installed --user
!pip install --quiet sentencepiece==0.1.94
!pip install --quiet tqdm==4.56.0
!pip install --quiet gradio

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account al

In [ ]:
# in case this cell does not run from the first time, re-run it

import torch

# in case the tensorflow module does not load => Restart Runtime => Load the libraries above again
import tensorflow as tf
from transformers import T5ForConditionalGeneration,T5Tokenizer
import gradio as gr

In [ ]:
# connect your personal google drive to load fine-tuned model
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Load the model

In [ ]:
# file path of trained model
trained_model_path = '/content/gdrive/My Drive/DISSERTATION/MODEL 1/t5/model/'

# file path of trained tokenizer
trained_tokenizer_path = '/content/gdrive/My Drive/DISSERTATION/MODEL 1/t5/tokenizer/'

In [ ]:
# load the trained model
model = T5ForConditionalGeneration.from_pretrained(trained_model_path)

# load the tokenizer
tokenizer = T5Tokenizer.from_pretrained(trained_tokenizer_path)

In [ ]:
# in case cuda.is_available() is True => then, the device used is a GPU
# otherwise, it's a CPU
# in our case is always GPU as it has been chosen in Runtime => Change runtime type
# if it was CPU, then the model would probably not be able to run at all
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# print what type of the device is used (GPU or CPU)
print ("device ",device)

# move the model to the device used (in my case is GPU)
model = model.to(device)

device  cuda


## Prepare Distractors

In [ ]:
def prepare_distractors(decoded_output):
    array_of_distractors_str = ""

    # starts from 1: => so, "distractors:" can be skipped
    for distractor in decoded_output.split()[1:]:
      array_of_distractors_str = array_of_distractors_str + distractor
    

    """ for some reason, BEAM DECODING METHOD adds an extra 'and' which needs to be removed """
    # remove 'and' => Example, ['Nothing','Never',and'More'] => ['Nothing','Never','More']
    array_of_distractors_str = array_of_distractors_str.replace(',and', ',')

    print("the distractors are: ", array_of_distractors_str)

    try:
      # convert string of distractors to a list of distractors
      # handle EOL string literal exception - answer input: supervisor => ['Supervisors','Supporter',"Supervisor']
      # handle this '] error of input fddsfsd => ['fddsfsd','Fddfsssdsdffsfdssffdftsdnffftdspds',['Fsdd']
      list_of_dist = eval(array_of_distractors_str.replace(',"', ",'").replace("'],", "',"))

      first_dist = list_of_dist[0]
      second_dist = list_of_dist[1]
      third_dist = list_of_dist[2]

    except IndexError: # in case there are less than 3 distractors => return 0 distractors
      return " ", " ", " "

    except: 
      # answer input: Cristiano Ronaldo => ['Ronaldo', 'Royal', ['Memphis']  - for some reason an extra '[' is added and we need to get rid of it
      array_of_distractors_str = array_of_distractors_str.replace("',['", "','")

      list_of_dist = eval(array_of_distractors_str)

      first_dist = list_of_dist[0]
      second_dist = list_of_dist[1]
      third_dist = list_of_dist[2]


    return first_dist, second_dist, third_dist

## Function for encoding the given answer by the user

In [ ]:
def encode_text(text):
    # encode text for feeding it to T5 model
    # the keys of the encoded dictionary are [input_ids, attention_mask]
    encoding = tokenizer.encode_plus(text,max_length =512, padding=True, return_tensors="pt")

    # move the encoded dictionary to the used device (GPU)
    input_ids  = encoding["input_ids"].to(device)

    return input_ids

## Decoding Strategies for Question Generation

### Beam Search

In [ ]:
def beam_search(input_ids):
    """
    Beam Search Decoding Strategy
    """

    # beams refer to the decoding style used - there are several kinds of decoding methods for generated2text models 
    beam_outputs = model.generate(
        input_ids=input_ids,  # the token ids of the the "text" variable above
        max_length=72,  # max length of the output 

        num_beams=3,  # 3 distractors
        no_repeat_ngram_size=3,  # no n-gram will appear three times => the ideal would be to be equal to 2 so no n-gram would appear twice but it ouputs errors
        num_return_sequences=1,  # generate one sequence of outputs/distractors
        early_stopping=True  # so that the generation is finished when all beam hypotheses reached the EOS token (</s>)
    )

    return beam_outputs

## Prepare GUI

In [ ]:
question = gr.inputs.Textbox(lines=2, placeholder="Enter question here...")
answer = gr.inputs.Textbox(lines=1, placeholder="Enter right answer here...")
word_embedding = gr.inputs.Radio(["Wordnet", "Sense2Vec"])

displayed_output = gr.outputs.HTML(label="question, right answer, distractors")

In [ ]:
def generate_distractors(question, answer):
    text = "question: "+ question + " " + "right answer: " + answer + " </s>"

    input_ids = encode_text(text)

    # place the model in evaluation mode => inference
    model.eval()


    # get the generated output
    generated_output = beam_search(input_ids) 


    # decode the generated output
    decoded_output = tokenizer.decode(generated_output[0], skip_special_tokens=True)


    # prepare distractors
    first_dist, second_dist, third_dist = prepare_distractors(decoded_output)


    # question +  \n + answer
    displayed_output = f"{question}<p> <b style='color:green;'> {answer} </b> </p>"

    # question + \n + answer + \n + 1st distractor + 2nd distractor + \n + 3rd distractor
    displayed_output = f"<p> {displayed_output} <b style='color:brown;'> {first_dist} </b> </p> \
                                                <b style='color:brown;'> {second_dist} </b> </p> \
                                                <b style='color:brown;'> {third_dist} </b> </p>"
    
    return displayed_output

In [ ]:
face = gr.Interface(
    fn = generate_distractors, 
    inputs = [question, answer], 
    outputs = displayed_output
)

## GUI 

In [ ]:
# if degug=True => prints the errors in the cell output
face.launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://25314.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:184: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1065: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


the distractors are:  ['Supervisors','Supporter',"Supervisor']
Keyboard interruption in main thread... closing server.


(<fastapi.applications.FastAPI at 0x7fe8ef2f7ad0>,
 'http://127.0.0.1:7860/',
 'https://25314.gradio.app')